In [13]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine-date')
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [14]:
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [15]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data,target,test_size= 0.2, random_state= 42)

In [16]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [17]:
print(sub_input.shape, val_input.shape,test_input.shape)

(4157, 3) (1040, 3) (1300, 3)


In [19]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input,sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input,val_target))

0.9971133028626413
0.864423076923077


In [20]:
#교차 검증
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.01399612, 0.01399231, 0.01400042, 0.01599979, 0.01308155]), 'score_time': array([0.00221348, 0.00200033, 0.00199866, 0.00295997, 0.00192142]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [21]:
#교차 검증의 최종 정ㅁ수
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [24]:
#타깃 클래스를 골고루 나눔
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [25]:
#10-폴드 교차 검증
splitter = StratifiedKFold(n_splits = 10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv = splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [26]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease':[0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [27]:
gs = GridSearchCV(DecisionTreeClassifier(random_state= 42),params,n_jobs=-1)

In [28]:
gs.fit(train_input,train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [29]:
dt = gs.best_estimator_
print(dt.score(train_input,train_target))

0.9615162593804117


In [30]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [31]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [32]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [39]:
params = {'min_impurity_decrease':np.arange(0.0001,0.001,0.0001),
         'max_depth': range(5,20,1),
         'min_samples_split':range(2,100,10)}

In [40]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1)
gs.fit(train_input,train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [41]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [42]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [43]:
from scipy.stats import uniform, randint

In [44]:
rgen = randint(0,10)
rgen.rvs(10)

array([8, 0, 3, 2, 3, 8, 8, 9, 7, 9])

In [45]:
np.unique(rgen.rvs(1000),return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 87, 125, 110,  91,  92, 110,  99,  84, 111,  91], dtype=int64))

In [55]:
params = {'min_impurity_decrease':uniform(0.0001,0.001),
         'max_depth':randint(20,50),
         'min_samples_split':randint(2,25),
         'min_samples_leaf':randint(1,25),}
params2 = {'min_impurity_decrease':uniform(0.0001,0.001),
         'max_depth':randint(20,50),
         'min_samples_split':randint(2,25),
         'min_samples_leaf':randint(1,25)}

print(params)
print(params2)#뭐가 다른거지?

{'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000023B0EC1AC88>, 'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000023B0EC1AF28>, 'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000023B0BB09C50>, 'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000023B0BB098D0>}
{'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000023B0EC1A048>, 'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000023B0BB09EF0>, 'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000023B0ED16668>, 'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000023B0ED165C0>}


In [61]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42,splitter='random'),params, n_iter=100,n_jobs=-1,random_state=42)
gs.fit(train_input,train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000023B0EC1AF28>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000023B0EC1AC88>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000023B0BB098D0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000023B0BB09C50>},
                   random_state=42)

In [62]:
print(gs.best_params_)

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}


In [63]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8458726956392981


In [60]:
dt = gs.best_estimator_
print(dt.score(test_input,test_target))
print(dt.score(train_input,train_target))

0.86
0.8928227823744468
